In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sympy import symbols, Function, diff

# Define a neural network structure based on HW2-2
class SimpleConvNet(nn.Module):
    def __init__(self):
        super(SimpleConvNet, self).__init__()
        self.conv = nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=0, bias=False)
        self.fc = nn.Linear(4, 1, bias=False)

    def forward(self, x):
        global R_matrix, O_matrix
        x = self.conv(x)
        x = F.sigmoid(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

model = SimpleConvNet()

input_feature_map = torch.randn(1, 1, 4, 4, requires_grad=True)
true_label = torch.tensor([[[17.]]])

output = model(input_feature_map)
loss_fn = nn.MSELoss()
loss = loss_fn(output, true_label)
print(loss)
print(model.conv.weight.grad)

model.zero_grad()
loss.backward()

gradient_weight_w11 = model.conv.weight.grad[0, 0, 1, 1]
gradient_input_a00 = input_feature_map.grad[0, 0, 0, 0]

print(f"Gradient with respect to w11: {gradient_weight_w11}")
print(f"Gradient of the input feature a00: {gradient_input_a00}")


tensor(295.1775, grad_fn=<MseLossBackward0>)
None
Gradient with respect to w11: -3.2137789726257324
Gradient of the input feature a00: -1.0458778142929077


c:\Users\User\anaconda3\envs\GPU\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([1, 1, 1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [3]:
def sigmoid(x):
    return 1 / (1 + torch.exp(-x))

def sigmoid_prime(x):
    return sigmoid(x) * (1 - sigmoid(x))

In [4]:

# w11

value = 2*(output - true_label)
input_aa = [
    input_feature_map[0][0][1][1], input_feature_map[0][0][1][2],
    input_feature_map[0][0][2][1], input_feature_map[0][0][2][2],
    ]
temp = 0
for ii in range(4):
    ww = model.fc.weight[0][ii]
    rr = model.conv(input_feature_map)[0][0][ii//2][ii%2]
    aa = input_aa[ii]
    temp += ww*sigmoid_prime(rr)*aa
gradient = value*temp
print(f"Manual calculated gradient of w11: {gradient.item()}")
print(f"Gradient with respect to conv w11: {gradient_weight_w11}")


Manual calculated gradient of w11: -3.213778495788574
Gradient with respect to conv w11: -3.2137789726257324


In [5]:
# a00

value = 2*(output - true_label)
wa = model.fc.weight[0][0]
rr = model.conv(input_feature_map)[0][0][0][0]
w00 = model.conv.weight[0][0][0][0]
# print(w00)
gradient_a00 = value * wa * sigmoid_prime(rr) * w00
print(f"Manual calculated gradient of a00: {gradient_a00.item()}")
print(f"Gradient of the input feature a00: {gradient_input_a00}")


Manual calculated gradient of a00: -1.0458778142929077
Gradient of the input feature a00: -1.0458778142929077


In [11]:
# w11

o, y, w11, a00, a01, a10, a11 = symbols('output yy w11 a00 a01 a10 a11')
w_fc = symbols('w_fc:4')
r = symbols('r:4')

sigmoid_prime = Function('sigmoidprime')

value = 2 * (o - y)
temp = sum(w_fc[ii] * sigmoid_prime(r[ii]) * symbols(f'a{ii//2}{ii%2}') for ii in range(4))
gradient = value * temp

gradient.simplify()


2*(output - yy)*(a00*w_fc0*sigmoidprime(r0) + a01*w_fc1*sigmoidprime(r1) + a10*w_fc2*sigmoidprime(r2) + a11*w_fc3*sigmoidprime(r3))

In [12]:
# a00

wa, w00 = symbols('wa w00')

gradient_a00 = 2 * (o - y) * wa * sigmoid_prime(r[0]) * w00

gradient_a00.simplify()


2*w00*wa*(output - yy)*sigmoidprime(r0)